In [ ]:
from lib.statistics import Statistics, stats_to_numpy, load_logs
import os
%pylab
%matplotlib inline

In [ ]:
log_folder_prefix = 'single_new_coh'
xlabel = "Weight of Cohesion (%)"

In [ ]:
"""
Parse data from logs
"""

paths = ["logs/" + path for path in os.listdir("logs") if path[:len(log_folder_prefix)] == log_folder_prefix]

data = {} # data = {cohesion : {"boid_count":array([simulations x measurements]), ""}}

for path in sorted(paths):
    ratio = float(path[5+len(log_folder_prefix):])
    
    pars, stats = load_logs(path)
    s, bc, sc, ss = stats_to_numpy(stats, pars)

    ratio_dic = {"boid_count" : bc[:, :-1], "school_count" : sc[:, :-1], "school_size" : ss[:, :-1]}

    data[ratio] = ratio_dic
    
print("Loaded in:", [np.round(a, 3) for a in data.keys()])

In [ ]:
"""
Bar plot displaying surviving distribution after 'cutoff' iterations
"""

# Process data
cutoff = 6500

surviving = []
surviving_std = []

xs = np.linspace(0, 1, 20, endpoint=True)

for i, (cohesion, cohesion_data) in enumerate(data.items()):
    if cohesion in xs:
        means = np.mean(cohesion_data["boid_count"], axis=0)
        stds = np.std(cohesion_data["boid_count"], axis=0)

        surviving.append(means[cutoff // pars.resolution])
        surviving_std.append(stds[cutoff // pars.resolution])

# Create plot
fig, ax = plt.subplots(figsize=(5,5))

ax.set_title("Survival rate of fish after " + str(cutoff) + " iterations")
ax.set_ylabel("Alive fish (% of total)")
ax.set_xlabel(xlabel)

ax.set_xlim(0,100)
ax.set_ylim(0,100)
for i in [20, 40, 60, 80, 100]:
    ax.axhline(i, color='gray', linestyle='dashed', alpha=0.1)
    
ax.bar(xs * 100, np.array(surviving)/250*100, yerr=np.array(surviving_std)/250*100, width=4, color="steelblue",
error_kw=dict(ecolor='black', lw=1, capsize=5, capthick=2))

plt.show()

In [ ]:
def percent_deviation(array, percent):
    """
    Returns the performance of the top percent%
    """
    rank = np.argsort(array.T)

    r = int(percent * len(array))
    result = array.T[np.arange(len(array.T)), rank[:, r]]
    
    return result

In [ ]:
"""
Line graph displaying the consistency and progression during simulations for different parameters
"""
labels = ["boid_count", "school_count"] 
scaling = {"boid_count":250/100, "school_count":1}

selected_ratios = [0.000, 0.263, 0.5, 0.737, 1]

band_percent_width = 0.341

for label in labels:
    fig, ax = plt.subplots(figsize=(14,7))

    color_map = plt.get_cmap("copper").reversed()
    sm = plt.cm.ScalarMappable(cmap=color_map)
    
    for i, (cohesion, cohesion_data) in enumerate(data.items()):
            if not selected_ratios:
                pass
            elif np.round(cohesion, 3) not in selected_ratios:
                continue
            
            array = cohesion_data[label]
                
            mean_school_progression = np.mean(array, axis=0)
            xs = np.arange(0, len(mean_school_progression)) * pars.resolution
            
            ax.plot(xs,
                    np.array(mean_school_progression)/scaling[label], color = color_map(i/len(paths)),
                    label=str(round(cohesion*100))+"%")
            
            
            ax.fill_between(xs,
                            percent_deviation(array, 0.5-band_percent_width)/scaling[label], 
                            percent_deviation(array, 0.5+band_percent_width)/scaling[label], 
                            alpha=0.1, color = color_map(i/len(paths)))
    if not selected_ratios:
        fig.colorbar(sm, label="cohesion influence over allignment influence")
    else:
        plt.legend()

    ax.set_title("Progress of " + label)

    ax.set_ylabel(label)

    ax.set_xlabel("Simulation Progress (%)")

    ax.set_xlim(0, max(xs))
    plt.show()